![Image Courtesy of: The Pokemon Company](img/International_Pokémon_logo.svg)
# PokeAPI Project
With the release of Pokemon Z-A, I felt it appropriate to create a project using one of the many pokemon APIs on the internet. One very interesting looking one, was [PokeAPI](pokeapi.co), which seems to offer a large variety of data related to pokemon, game items and locations. With that being said, I went ahead and started creating a MariaDB database to work with for this project.

## Connection to MariaDB
First, we create a connection to an already existing database. This database is hosted on a local MariaDB server, and can be viewed locally using DBeaver as well.

![view_in_dbeaver](img/db_in_dbeaver.jpg)

In [34]:
import os, mysql.connector
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
import requests
import json
import time
from datetime import datetime

load_dotenv()

cfg = dict(
    host=os.getenv("MYSQL_HOST"),
    port=int(os.getenv("MYSQL_PORT", 3306)),
    user=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASSWORD"),
    database=os.getenv("MYSQL_DB"),
)

conn = mysql.connector.connect(**cfg)
cur = conn.cursor()
cur.execute("SELECT DATABASE();")
engine = create_engine(f"mysql+mysqlconnector://{cfg['user']}:{cfg['password']}@{cfg['host']}:{cfg['port']}/{cfg['database']}")

print("Connected to database:", cur.fetchone()[0])

Connected to database: pokeapi_project


Now that we are connected to the database, we will soon need to populate it with data. Since we have not yet seen what the responses from the API will look like, we'll create a raw landing table for the API data in the meantime, while we look at a sample.

In [35]:

create_sql = """
CREATE TABLE IF NOT EXISTS poke_raw (
    id BIGINT AUTO_INCREMENT PRIMARY KEY,
    resource VARCHAR(40) NOT NULL,
    external_id VARCHAR(40) NOT NULL,
    payload JSON NOT NULL,
    fetched_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UNIQUE KEY u_resource_external (resource, external_id),
    KEY idx_resource(resource)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
"""

cur.execute(create_sql)
conn.commit()
print("Table 'poke_raw' created successfully.")


Table 'poke_raw' created successfully.


We created the table successfully, so now it's time to make the first call and take a look at the data. This specific API offers a wide variety of endpoints, so we have lots of data  to choose from as well as mix and match data from different endpoints for different focuses and insights. First, we'll see what the `pokemon` endpoint offers. 

According to the API documentation, there are different endpoints under the `pokemon` group. The ones we will target are `pokemon-species`, `ability`, `pokemon-form`, and `pokemon-habitat`, which should return JSON responses holding general information about a pokemon or ability, given the passed `id`.

In [36]:
import requests, json
BASE = "https://pokeapi.co/api/v2/"

def get_species(id):
    """Fetch pokemon species by ID from PokeAPI."""
    req = requests.get(f"{BASE}pokemon-species/{id}/", timeout=30)
    req.raise_for_status()
    return req.json()

def pick_en(entries, value_key):
    """Pick English entry from a list of entries."""
    for e in entries:
        lang = (e.get("language") or {}).get("name")
        if lang == "en":
            return e.get(value_key)
    return None

# fetch one species
sp = get_species(6)

# build a compact summary of fields to normalize lateer
summary = {
    "id": sp.get("id"),
    "name": sp.get("name"),
    "order": sp.get("order"),
    "is_baby": sp.get("is_baby"),
    "is_legendary": sp.get("is_legendary"),
    "is_mythical": sp.get("is_mythical"),
    "gender_rate": sp.get("gender_rate"),
    "capture_rate": sp.get("capture_rate"),
    "base_happiness": sp.get("base_happiness"),
    "hatch_counter": sp.get("hatch_counter"),
    "forms_switchable": sp.get("forms_switchable"),
    "growth_rate": (sp.get("growth_rate") or {}).get("name"),
    "color": (sp.get("color") or {}).get("name"),
    "shape": (sp.get("shape") or {}).get("name"),
    "habitat": (sp.get("habitat") or {}).get("name"),
    "generation": (sp.get("generation") or {}).get("name"),
    "evolves_from_species": (sp.get("evolves_from_species") or {}).get("name"),
    "evolution_chain_id": (lambda url: int(url.rstrip("/").split("/")[-1]) if url else None)(
        (sp.get("evolution_chain") or {}).get("url")
    ),
    "egg_groups": [eg.get("name") for eg in (sp.get("egg_groups") or [])],
    "pokedex_numbers": [
        {
            "entry_number": pn.get("entry_number"),
            "pokedex": (pn.get("pokedex") or {}).get("name")
        }
        for pn in (sp.get("pokedex_numbers") or [])
    ],
    "genus_en": pick_en(sp.get("genera"), "genus"),
    "flavor_text_en": (pick_en(sp.get("flavor_text_entries"), "flavor_text") or "").replace("\n"," ").replace("\f"," ").strip(),
}

print("===pokemon species sumary (id=6) ===")
print(json.dumps(summary, indent=2))


===pokemon species sumary (id=6) ===
{
  "id": 6,
  "name": "charizard",
  "order": 6,
  "is_baby": false,
  "is_legendary": false,
  "is_mythical": false,
  "gender_rate": 1,
  "capture_rate": 45,
  "base_happiness": 70,
  "hatch_counter": 20,
  "forms_switchable": true,
  "growth_rate": "medium-slow",
  "color": "red",
  "shape": "upright",
  "habitat": "mountain",
  "generation": "generation-i",
  "evolves_from_species": "charmeleon",
  "evolution_chain_id": 2,
  "egg_groups": [
    "monster",
    "dragon"
  ],
  "pokedex_numbers": [
    {
      "entry_number": 6,
      "pokedex": "national"
    },
    {
      "entry_number": 6,
      "pokedex": "kanto"
    },
    {
      "entry_number": 231,
      "pokedex": "original-johto"
    },
    {
      "entry_number": 236,
      "pokedex": "updated-johto"
    },
    {
      "entry_number": 111,
      "pokedex": "conquest-gallery"
    },
    {
      "entry_number": 85,
      "pokedex": "kalos-central"
    },
    {
      "entry_number": 6,
  

>Great, we have successfully fetched data from the first endpoint, `pokemon-species` endpoint. We will use this data to now populate the first table, called `species_raw`, within the database. The table we previously created, called `poke_raw`, won't be of much use. We could store the data there as a JSON column, but that would mean having to clean it up and extracting the information we want every time we want to explore that table. Instead, in the new `species_raw` table, we'll clean the data first and add it to that table in separate columns, having a much cleaner database to work with.

In [37]:
import os, mysql.connector

cur.execute("""
CREATE TABLE IF NOT EXISTS species_raw (
    id BIGINT AUTO_INCREMENT PRIMARY KEY,
    external_id INT NOT NULL,            -- species numeric id
    name VARCHAR(80) NOT NULL,           -- species identifier (API 'name')
    payload JSON NOT NULL,               -- raw JSON
    fetched_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UNIQUE KEY u_species (external_id),
    KEY idx_name (name)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
""")
conn.commit()
print("species_raw ready")

sid = int(sp["id"])
nm = sp["name"]
payload_text = json.dumps(sp, separators=(',', ':'))

cur.execute("""
INSERT INTO species_raw (external_id, name, payload)
VALUES (%s, %s, %s)
ON DUPLICATE KEY UPDATE
    name=VALUES(name),
    payload=VALUES(payload),
    fetched_at=CURRENT_TIMESTAMP
""", (sid, nm, payload_text))
conn.commit()

# show confirmation row count
cur.execute("SELECT COUNT(*) FROM species_raw;")
print("species_raw rows:", cur.fetchone()[0])

species_raw ready
species_raw rows: 1025


>Now that we have created the table and know the pipeline works, we can now hit the API with requests for all the pokemon species via the official paged list, and follow each item's url.

In [38]:
import requests, json, time
BASE = "https://pokeapi.co/api/v2/"

def list_species(limit=100, offset=0):
    """List pokemon species from PokeAPI with pagination."""
    req = requests.get(f"{BASE}pokemon-species/", params={"limit": limit, "offset": offset}, timeout=30)
    req.raise_for_status()
    return req.json()

def fetch_url(url:str) -> dict:
    """Fetch JSON data from a given URL."""
    req = requests.get(url, timeout=30)
    req.raise_for_status()
    return req.json()

def upsert_species_raw(doc:dict):
    """Insert or update species_raw table with given document."""
    sid = int(doc["id"])
    nm = doc["name"]
    payload_text = json.dumps(doc, separators=(',', ':'))

    cur.execute("""
    INSERT INTO species_raw (external_id, name, payload)
    VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE
        name=VALUES(name),
        payload=VALUES(payload),
        fetched_at=CURRENT_TIMESTAMP
    """, (sid, nm, payload_text))

meta = list_species(limit=1, offset=0)
total = meta["count"]
print(f"Total species to fetch: {total}")

# skip already seen species
cur.execute("SELECT external_id FROM species_raw;")
existing = {row[0] for row in cur.fetchall()}
print(f"Already existing species in DB: {len(existing)}, will only fetch missing ones.")

# page through the API
page = 100
inserted = 0
for offset in range(0, total, page):
    page_data = list_species(limit=page, offset=offset)
    for item in page_data["results"]:
        doc = fetch_url(item["url"])
        if int(doc["id"]) in existing:
            continue
        upsert_species_raw(doc)
        inserted += 1
        # rate limiting
        time.sleep(0.05)
    conn.commit()
    print(f"Fetched and inserted {inserted} new species so far...")

# final count
cur.execute("SELECT COUNT(*) FROM species_raw;")
print("Final species_raw rows:", cur.fetchone()[0])

Total species to fetch: 1025
Already existing species in DB: 1025, will only fetch missing ones.
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Fetched and inserted 0 new species so far...
Final species_raw rows: 1025


Perfect, now we know that there are a total of 1025 pokemon available on the API. With this information we can proceed to build a clean schema and tables that we can use for the rest of the project, and proceed to data exploration afterwards.

In [39]:
# Create clean schema and tables based on the fetched data

ddl_statements = [
    # lookups
    """
    CREATE TABLE IF NOT EXISTS growth_rate (
      id INT AUTO_INCREMENT PRIMARY KEY,
      name VARCHAR(50) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """,
    """
    CREATE TABLE IF NOT EXISTS color (
      id INT AUTO_INCREMENT PRIMARY KEY,
      name VARCHAR(50) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """,
    """
    CREATE TABLE IF NOT EXISTS habitat (
      id INT AUTO_INCREMENT PRIMARY KEY,
      name VARCHAR(50) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """,
    """
    CREATE TABLE IF NOT EXISTS shape (
      id INT AUTO_INCREMENT PRIMARY KEY,
      name VARCHAR(50) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """,
    """
    CREATE TABLE IF NOT EXISTS generation (
      id INT AUTO_INCREMENT PRIMARY KEY,
      name VARCHAR(50) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """,
    """
    CREATE TABLE IF NOT EXISTS egg_group (
      id INT AUTO_INCREMENT PRIMARY KEY,
      name VARCHAR(50) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """,

    # main species table
    """
    CREATE TABLE IF NOT EXISTS species (
      id INT PRIMARY KEY,
      name VARCHAR(80) NOT NULL UNIQUE,

      -- english-facing text
      genus_en VARCHAR(120),
      flavor_text_en TEXT,

      -- flags & scalar attributes
      is_baby TINYINT(1),
      is_legendary TINYINT(1),
      is_mythical TINYINT(1),
      is_default TINYINT(1),
      gender_rate INT,
      capture_rate INT,
      base_happiness INT,
      hatch_counter INT,
      forms_switchable TINYINT(1),

      -- lookups
      growth_rate_id INT,
      color_id INT,
      habitat_id INT,
      shape_id INT,
      generation_id INT,

      -- lineage references
      evolves_from_species VARCHAR(80),
      evolution_chain_id INT,

      -- FKs
      CONSTRAINT fk_species_growth_rate FOREIGN KEY (growth_rate_id) REFERENCES growth_rate(id),
      CONSTRAINT fk_species_color       FOREIGN KEY (color_id)       REFERENCES color(id),
      CONSTRAINT fk_species_habitat     FOREIGN KEY (habitat_id)     REFERENCES habitat(id),
      CONSTRAINT fk_species_shape       FOREIGN KEY (shape_id)       REFERENCES shape(id),
      CONSTRAINT fk_species_generation  FOREIGN KEY (generation_id)  REFERENCES generation(id)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """,

    # many-to-many: species - egg_group
    """
    CREATE TABLE IF NOT EXISTS species_egg_group (
      species_id INT,
      egg_group_id INT,
      PRIMARY KEY (species_id, egg_group_id),
      CONSTRAINT fk_seg_species    FOREIGN KEY (species_id)   REFERENCES species(id),
      CONSTRAINT fk_seg_egg_group  FOREIGN KEY (egg_group_id) REFERENCES egg_group(id)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """,

    # helpful indexes for analysis
    "CREATE INDEX IF NOT EXISTS idx_species_color       ON species(color_id);",
    "CREATE INDEX IF NOT EXISTS idx_species_habitat     ON species(habitat_id);",
    "CREATE INDEX IF NOT EXISTS idx_species_generation  ON species(generation_id);",
    "CREATE INDEX IF NOT EXISTS idx_species_growth_rate ON species(growth_rate_id);"
]

for stmt in ddl_statements:
    cur.execute(stmt)
conn.commit()
print("Clean schema ready: growth_rate, color, habitat, shape, generation, egg_group, species, species_egg_group.")

Clean schema ready: growth_rate, color, habitat, shape, generation, egg_group, species, species_egg_group.


Once we have those created, we can proceed to fill them, using data from species_raw to do so:

In [40]:
import json
from sqlalchemy import text

def get_en(items, value_key, lang_key="language", lang="en"):
    """Return the first English entry's value (e.g., flavor text, genus)."""
    for it in items or []:
        if ((it.get(lang_key) or {}).get("name")) == lang:
            return it.get(value_key)
    return None

def parse_chain_id(url: str|None):
    if not url:
        return None
    try:
        return int(url.rstrip("/").split("/")[-1])
    except Exception:
        return None

def upsert_lookup(table: str, name: str|None):
    """Insert name into a lookup table if needed and return its id (or None)."""
    if not name:
        return None
    cur.execute(f"INSERT IGNORE INTO {table} (name) VALUES (%s)", (name,))
    cur.execute(f"SELECT id FROM {table} WHERE name=%s", (name,))
    row = cur.fetchone()
    return row[0] if row else None

# pull all species_raw payloads
rows = pd.read_sql(text("SELECT external_id, payload FROM species_raw ORDER BY external_id"), engine)
print("Transforming", len(rows), "species")

batch = 0
for _, r in rows.iterrows():
    s = json.loads(r["payload"])

    # lookups
    growth_rate_id = upsert_lookup("growth_rate", (s.get("growth_rate") or {}).get("name"))
    color_id       = upsert_lookup("color",       (s.get("color") or {}).get("name"))
    habitat_id     = upsert_lookup("habitat",     (s.get("habitat") or {}).get("name"))
    shape_id       = upsert_lookup("shape",       (s.get("shape") or {}).get("name"))
    generation_id  = upsert_lookup("generation",  (s.get("generation") or {}).get("name"))

    # english texts
    genus_en   = get_en(s.get("genera"), "genus")
    flavor_en  = get_en(s.get("flavor_text_entries"), "flavor_text") or ""
    flavor_en  = flavor_en.replace("\n", " ").replace("\f", " ").strip()

    # lineage + flags
    evo_chain_id = parse_chain_id((s.get("evolution_chain") or {}).get("url"))
    evolves_from = (s.get("evolves_from_species") or {}).get("name")

    # upsert species
    cur.execute("""
        INSERT INTO species (
            id, name, genus_en, flavor_text_en,
            is_baby, is_legendary, is_mythical, is_default,
            gender_rate, capture_rate, base_happiness, hatch_counter, forms_switchable,
            growth_rate_id, color_id, habitat_id, shape_id, generation_id,
            evolves_from_species, evolution_chain_id
        ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE
            name=VALUES(name),
            genus_en=VALUES(genus_en),
            flavor_text_en=VALUES(flavor_text_en),
            is_baby=VALUES(is_baby),
            is_legendary=VALUES(is_legendary),
            is_mythical=VALUES(is_mythical),
            is_default=VALUES(is_default),
            gender_rate=VALUES(gender_rate),
            capture_rate=VALUES(capture_rate),
            base_happiness=VALUES(base_happiness),
            hatch_counter=VALUES(hatch_counter),
            forms_switchable=VALUES(forms_switchable),
            growth_rate_id=VALUES(growth_rate_id),
            color_id=VALUES(color_id),
            habitat_id=VALUES(habitat_id),
            shape_id=VALUES(shape_id),
            generation_id=VALUES(generation_id),
            evolves_from_species=VALUES(evolves_from_species),
            evolution_chain_id=VALUES(evolution_chain_id)
    """, (
        s["id"],
        s.get("name"),
        genus_en,
        flavor_en,
        int(bool(s.get("is_baby"))),
        int(bool(s.get("is_legendary"))),
        int(bool(s.get("is_mythical"))),
        int(bool(s.get("is_default"))),
        s.get("gender_rate"),
        s.get("capture_rate"),
        s.get("base_happiness"),
        s.get("hatch_counter"),
        int(bool(s.get("forms_switchable"))),
        growth_rate_id,
        color_id,
        habitat_id,
        shape_id,
        generation_id,
        evolves_from,
        evo_chain_id
    ))

    # species - egg_groups bridge (reset then insert)
    cur.execute("DELETE FROM species_egg_group WHERE species_id=%s", (s["id"],))
    for eg in (s.get("egg_groups") or []):
        eg_id = upsert_lookup("egg_group", eg.get("name"))
        if eg_id:
            cur.execute("""
                INSERT IGNORE INTO species_egg_group (species_id, egg_group_id)
                VALUES (%s, %s)
            """, (s["id"], eg_id))

    batch += 1
    if batch % 200 == 0:
        conn.commit()
        print(f"Committed {batch} species…")

conn.commit()
print(f"Done. Transformed {batch} species into clean tables.")


Transforming 1025 species
Committed 200 species…
Committed 400 species…
Committed 600 species…
Committed 800 species…
Committed 1000 species…
Done. Transformed 1025 species into clean tables.


A quick sanity check to see where we're at

In [41]:
import pandas as pd
from sqlalchemy import text

# total rows
print("Row counts:")
for tbl in ["species", "growth_rate", "color", "habitat", "shape", "generation", "egg_group", "species_egg_group"]:
    try:
        n = pd.read_sql(text(f"SELECT COUNT(*) AS n FROM {tbl};"), engine).iloc[0]["n"]
        print(f"  {tbl:18s} -> {n}")
    except Exception as e:
        print(f"  {tbl:18s} -> (missing?) {e}")

# head of species (core columns)
preview = pd.read_sql(text("""
SELECT s.id, s.name, s.genus_en,
       s.is_legendary, s.is_mythical, s.is_baby,
       gr.name AS growth_rate, c.name AS color,
       h.name AS habitat, sh.name AS shape, g.name AS generation
FROM species s
LEFT JOIN growth_rate gr ON gr.id = s.growth_rate_id
LEFT JOIN color       c  ON c.id  = s.color_id
LEFT JOIN habitat     h  ON h.id  = s.habitat_id
LEFT JOIN shape       sh ON sh.id = s.shape_id
LEFT JOIN generation  g  ON g.id  = s.generation_id
ORDER BY s.id
LIMIT 10;
"""), engine)
preview


Row counts:
  species            -> 1025
  growth_rate        -> 6
  color              -> 10
  habitat            -> 9
  shape              -> 14
  generation         -> 9
  egg_group          -> 15
  species_egg_group  -> 1304


,id,name,genus_en,is_legendary,is_mythical,is_baby,growth_rate,color,habitat,shape,generation
0,1,bulbasaur,Seed Pokémon,0,0,0,medium-slow,green,grassland,quadruped,generation-i
1,2,ivysaur,Seed Pokémon,0,0,0,medium-slow,green,grassland,quadruped,generation-i
2,3,venusaur,Seed Pokémon,0,0,0,medium-slow,green,grassland,quadruped,generation-i
3,4,charmander,Lizard Pokémon,0,0,0,medium-slow,red,mountain,upright,generation-i
4,5,charmeleon,Flame Pokémon,0,0,0,medium-slow,red,mountain,upright,generation-i
5,6,charizard,Flame Pokémon,0,0,0,medium-slow,red,mountain,upright,generation-i
6,7,squirtle,Tiny Turtle Pokémon,0,0,0,medium-slow,blue,waters-edge,upright,generation-i
7,8,wartortle,Turtle Pokémon,0,0,0,medium-slow,blue,waters-edge,upright,generation-i
8,9,blastoise,Shellfish Pokémon,0,0,0,medium-slow,blue,waters-edge,upright,generation-i
9,10,caterpie,Worm Pokémon,0,0,0,medium,green,forest,armor,generation-i


Since everything looks good, we can now proceed to store the flattened `species_raw` table as `species_cleaned` before downloading any more data.

In [42]:
import json
import pandas as pd
from sqlalchemy import text

cur.execute("""
CREATE TABLE IF NOT EXISTS species_clean (
  id INT PRIMARY KEY,
  name VARCHAR(80) NOT NULL UNIQUE,
  genus_en VARCHAR(120),
  flavor_text_en TEXT,

  -- flags
  is_baby TINYINT(1),
  is_legendary TINYINT(1),
  is_mythical TINYINT(1),
  is_default TINYINT(1),

  -- numeric attributes
  gender_rate INT,
  capture_rate INT,
  base_happiness INT,
  hatch_counter INT,
  forms_switchable TINYINT(1),

  -- taxonomy as plain text (flattened lookups)
  growth_rate VARCHAR(50),
  color       VARCHAR(50),
  habitat     VARCHAR(50),
  shape       VARCHAR(50),
  generation  VARCHAR(50),

  -- lineage
  evolves_from_species VARCHAR(80),
  evolution_chain_id INT
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
""")
conn.commit()

def get_en(items, value_key, lang_key="language", lang="en"):
    for it in items or []:
        if ((it.get(lang_key) or {}).get("name")) == lang:
            return it.get(value_key)
    return None

def parse_chain_id(url: str | None):
    if not url: return None
    try: return int(url.rstrip("/").split("/")[-1])
    except: return None

# 2) read all raw payloads
rows = pd.read_sql(text("SELECT external_id, payload FROM species_raw ORDER BY external_id"), engine)
print("Transforming rows:", len(rows))

# 3) upsert into species_clean
processed = 0
for _, r in rows.iterrows():
    s = json.loads(r["payload"])

    genus_en  = get_en(s.get("genera"), "genus")
    flavor_en = (get_en(s.get("flavor_text_entries"), "flavor_text") or "").replace("\n"," ").replace("\f"," ").strip()

    cur.execute("""
        INSERT INTO species_clean (
          id, name, genus_en, flavor_text_en,
          is_baby, is_legendary, is_mythical, is_default,
          gender_rate, capture_rate, base_happiness, hatch_counter, forms_switchable,
          growth_rate, color, habitat, shape, generation,
          evolves_from_species, evolution_chain_id
        ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE
          name=VALUES(name),
          genus_en=VALUES(genus_en),
          flavor_text_en=VALUES(flavor_text_en),
          is_baby=VALUES(is_baby),
          is_legendary=VALUES(is_legendary),
          is_mythical=VALUES(is_mythical),
          is_default=VALUES(is_default),
          gender_rate=VALUES(gender_rate),
          capture_rate=VALUES(capture_rate),
          base_happiness=VALUES(base_happiness),
          hatch_counter=VALUES(hatch_counter),
          forms_switchable=VALUES(forms_switchable),
          growth_rate=VALUES(growth_rate),
          color=VALUES(color),
          habitat=VALUES(habitat),
          shape=VALUES(shape),
          generation=VALUES(generation),
          evolves_from_species=VALUES(evolves_from_species),
          evolution_chain_id=VALUES(evolution_chain_id)
    """, (
        s["id"], s.get("name"), genus_en, flavor_en,
        int(bool(s.get("is_baby"))),
        int(bool(s.get("is_legendary"))),
        int(bool(s.get("is_mythical"))),
        int(bool(s.get("is_default"))),
        s.get("gender_rate"),
        s.get("capture_rate"),
        s.get("base_happiness"),
        s.get("hatch_counter"),
        int(bool(s.get("forms_switchable"))),
        (s.get("growth_rate") or {}).get("name"),
        (s.get("color") or {}).get("name"),
        (s.get("habitat") or {}).get("name"),
        (s.get("shape") or {}).get("name"),
        (s.get("generation") or {}).get("name"),
        (s.get("evolves_from_species") or {}).get("name"),
        parse_chain_id((s.get("evolution_chain") or {}).get("url"))
    ))
    processed += 1
    if processed % 200 == 0:
        conn.commit()
        print(f"Committed {processed}…")

conn.commit()
print(f"Done. species_clean rows upserted: {processed}")

# quick sanity
cur.execute("SELECT COUNT(*) FROM species_clean;")
print("species_clean total:", cur.fetchone()[0])

Transforming rows: 1025
Committed 200…
Committed 400…
Committed 600…
Committed 800…
Committed 1000…
Done. species_clean rows upserted: 1025
species_clean total: 1025


## NOTE
>Using a local MariaDB setup has been successful, but for the purposes of this project, the work will be migrated to a MongoDB cloud database. That can be seen in `pokeapi_to_mongodb.ipynb`.

## Sources

* [Free API Lst on Github](https://github.com/public-api-lists)
* [Poke API](https://pokeapi.co/)